# Advent of Code 2024

# Puzzle - part 1

**--- Day 2: Red-Nosed Reports ---**

Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office.

While the *Red-Nosed Reindeer nuclear fusion/fission plant* appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they **still** talk about the time Rudolph was saved through molecular synthesis from a single electron.

They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data.

The unusual data (your puzzle input) consists of many **reports**, one report per line. Each report is a list of numbers called **levels** that are separated by spaces. For example:

```
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
```

This example data contains six reports each containing five levels.

The engineers are trying to figure out which reports are **safe**. The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true:

- The levels are either **all increasing** or **all decreasing**.
- Any two adjacent levels differ by **at least one** and **at most three**.
  
In the example above, the reports can be found safe or unsafe by checking those rules:

- `7 6 4 2 1`: Safe because the levels are all decreasing by `1` or `2`.
- `1 2 7 8 9`: Unsafe because `2` `7` is an increase of `5`.
- `9 7 6 2 1`: Unsafe because `6` `2` is a decrease of `4`.
- `1 3 2 4 5`: Unsafe because `1` `3` is increasing but `3` `2` is decreasing.
- `8 6 4 4 1`: Unsafe because `4` `4` is neither an increase or a decrease.
- `1 3 6 7 9`: Safe because the levels are all increasing by `1`, `2`, or `3`.
  
So, in this example, `2` reports are safe.

Analyze the unusual data from the engineers.
**How many reports are safe?**

## Input

In [90]:
# Load the input file

with open('input - Day 2.txt', 'r') as file:
    input = file.read()


print(input[:139])

44 47 50 51 53 54 53
70 73 75 77 80 81 84 84
1 3 4 7 10 13 16 20
47 49 52 53 55 57 60 65
69 70 71 70 71
22 23 20 21 24 27 24
90 92 93 94 95


## Input Formatting

Right now we just have a text file with values from both lists. What we want is two arrays each containing values from only one list. First let us separate each line so that we can later iterate over each line.

In [91]:
from pprint import pprint

input_lines = input.split('\n')
pprint(input_lines[:10])

print(f"\nThere are {len(input_lines)} lines")

['44 47 50 51 53 54 53',
 '70 73 75 77 80 81 84 84',
 '1 3 4 7 10 13 16 20',
 '47 49 52 53 55 57 60 65',
 '69 70 71 70 71',
 '22 23 20 21 24 27 24',
 '90 92 93 94 95 93 94 94',
 '16 18 15 16 20',
 '47 48 51 50 55',
 '27 28 31 31 32 34']

There are 1001 lines


In [92]:
# Check the last line, if it's empty then let's remove it
if input_lines[-1] == "":
    del input_lines[-1]

print(len(input_lines))

1000


We are going to change each report to a list of levels so that we can iterate over them.
Gotta convert them to integers while we are at it.

In [93]:
from pprint import pprint
import re

all_reports = []

for line in input_lines:
    levels = re.split(r"\s+", line)

    levels = [int(level) for level in levels]
    all_reports.append(levels)

pprint(all_reports[:10])

[[44, 47, 50, 51, 53, 54, 53],
 [70, 73, 75, 77, 80, 81, 84, 84],
 [1, 3, 4, 7, 10, 13, 16, 20],
 [47, 49, 52, 53, 55, 57, 60, 65],
 [69, 70, 71, 70, 71],
 [22, 23, 20, 21, 24, 27, 24],
 [90, 92, 93, 94, 95, 93, 94, 94],
 [16, 18, 15, 16, 20],
 [47, 48, 51, 50, 55],
 [27, 28, 31, 31, 32, 34]]


## Solution

Well this time there really is no way around it. 
We have to use nested loops, thankfully none of the reports are very long so we don't really care that it's **O(n^2)**.

We will loop over **all reports** and then for each report subtract the two values and check if the change is okay.
We will need to track the last change to know if it's **decreasing** or **increasing**.

Each raport will be marked **safe** or **unsafe** in a separate list.


In [94]:
def sign(number): # Returns True for positive numbers and False for negative numbers
    if number < 0:
        return False
    else:
       return True

In [95]:
analyzed_reports = [] # 0 = unsafe | 1 = safe

for report in all_reports:

    safety = 1 # 0 = unsafe | 1 = safe

    last_change_sign = None
    last_level = None

    for level in report:

        # Skip if on the first level
        if last_level is None:
            last_level = level
            continue

        # Second and later levels
        change = level - last_level
        last_level = level # Don't forget to update last_level before you proceed

        # Change if the magnitude of the change is not safe.
        # This might exit the loop so make sure you are done with everything else.
        # For example updating last_level (╯°□°)╯︵ ┻━┻
        if abs(change) < 1 or abs(change) > 3:
            safety = 0 # Unsafe
            break
        
        # If on second level set last_change_sign
        if last_change_sign is None:
            last_change_sign = sign(change)
            continue
            

        if sign(change) != last_change_sign:
            safety = 0 # Unsafe
            break
            
            
    # If all levels passed all checks then the report is safe
    analyzed_reports.append(safety)

No just add up all the values in `safety_levels` to know how many **reports** are **safe**.

In [96]:
print(f"Safe reports: {sum(analyzed_reports)}")

Safe reports: 591


I will be honest here, this took me way more time than I though it would.
The mistake I made was `last_change_sign` is a **boolean**, so I can't just check if the value exists like so with `if last_change_sign:` because then I am actually checking the value of the boolean. Lesson learnt, use `if last_change_sign is None` from now on.

# Puzzle - Part 2


**--- Part Two ---**

The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the Problem Dampener.

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems **tolerate a single bad level** in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

- `7 6 4 2 1`: Safe without removing any level.
- `1 2 7 8 9`: Unsafe regardless of which level is removed.
- `9 7 6 2 1`: Unsafe regardless of which level is removed.
- `1 3 2 4 5`: Safe by removing the second level, `3`.
- `8 6 4 4 1`: Safe by removing the third level, `4`.
- `1 3 6 7 9`: Safe without removing any level.

Thanks to the Problem Dampener, `4` reports are actually safe!

Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports.
**How many reports are now safe?**

______________________________

Okay so this will require a re-design but we can re-use some of the work from before.
What we need is two things:

- Identify if a report is **safe**.
- Identify the numbers responsible for an unsafe change.

Once we know these things we can remove each of the unsafe number are re-calculare the report until we have tried removing all unsafe numbers one by one.

For a change in magnitude we only need to *mark* two numbers, but for a change in sign we need to add all three numbers.
That is because if we have a series like this `n1 -> n2 -> n3` then the previous sign (+/-) has two components `n1` and `n2`, while the current sign's components are `n2` and `n3`.
So we have to mark all three as potentially unsafe.


In [97]:
def check_report_safety(report):
    unsafe_indicies = set() # Using a set to avoid duplicates

    safety = 1 # 0 = unsafe | 1 = safe

    last_change_sign = None
    last_level = None

    for idx, level in enumerate(report):

        # Skip if on the first level
        if last_level is None:
            last_level = level
            continue

        # Second and later levels
        change = level - last_level
        last_level = level # Don't forget to update last_level before you proceed

        # Change if the magnitude of the change is not safe.
        # This might exit the loop so make sure you are done with everything else.
        # For example updating last_level (╯°□°)╯︵ ┻━┻
        if abs(change) < 1 or abs(change) > 3:
            safety = 0 # Unsafe
            unsafe_indicies.add(idx)
            unsafe_indicies.add(idx-1)
            continue
        
        # If on second level set last_change_sign
        if last_change_sign is None:
            last_change_sign = sign(change)
            continue
            
        if sign(change) != last_change_sign:
            safety = 0 # Unsafe
            unsafe_indicies.add(idx)
            unsafe_indicies.add(idx-1)
            # The sign change is actually an effect of 3 numbers
            # |n1 <-> n2 <-> n3| so we need to add all 3 as unsafe
            unsafe_indicies.add(abs(idx-2))

            continue
            
            
    # If all levels passed all checks then the report is safe
    return safety, unsafe_indicies

In [98]:
import copy

analyzed_reports = []

for report in all_reports:
    
    safety, unsafe_indicies = check_report_safety(report)

    # if safe
    if safety == 1:
        analyzed_reports.append(1)

    else:
        safety_with_dampener = 0 # 0 -> unsafe | 1 -> safe

        for unsafe_idx in unsafe_indicies:

            dampened_report = copy.deepcopy(report) # A deep copy - a new list is created
            del dampened_report[unsafe_idx] # Removing the unsafe index from the copy

            dampened_safety, _ = check_report_safety(dampened_report)
            
            # If the new report is safe with a removed entry
            if dampened_safety == 1:
                safety_with_dampener = 1
                break
        
        analyzed_reports.append(safety_with_dampener)

In [99]:
print(f"Safe reports: {sum(analyzed_reports)}")

Safe reports: 621
